# Cuaderno 2: Problema de asignación óptima

A menudo las variables y restricciones de un modelo de programación lineal (entera) dependen de índices que representan elementos de conjuntos finitos.

Un ejemplo clásico de un problema que puede ser formulado como un programa lineal entero es el **problema de asignación óptima** (*assignment problem*).

Se tienen dados un conjunto $I$ de personas y un conjunto $J$ de tareas. Se ha medido la afinidad de cada persona $i \in I$ para desempeñar cada tarea $j \in J$, expresando este valor a través de una calificación $c_{ij}$, donde una calificación más alta indica una mayor afinidad (es decir, una mayor destreza en el desempeño de la tarea por parte de la persona). Se desea asignar cada tarea a una persona, de tal forma que se maximice la suma de las calificaciones correspondientes a las asignaciones realizadas. Ninguna tarea puede ser asignada a más de una persona, y ninguna persona puede tener más de una tarea asignada.

El problema de encontrar una asignación óptima puede ser formulado como el siguiente programa lineal entero.

\begin{align*}
&\max \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\ 
&\mbox{s.r.}\\
& \sum_{j \in J} x_{ij} \leq 1, \quad \forall i \in I, \\
& \sum_{i \in I} x_{ij} \leq 1, \quad \forall j \in J, \\
& x_{ij} \in \{0, 1\}, \quad \forall i \in I, j \in J.
\end{align*}

Cada variable binaria de decisión $x_{ij}$ indica si una persona $i \in I$ ha sido asignada a una tarea $j \in J$. La función objetivo mide la calificación total de todas las asignaciones realizadas. 

La primera familia de restricciones expresa que ninguna persona puede tener más de una tarea asignada. Observar que existe una restricción por cada persona.

La segunda familia de restricciones expresa que ninguna tarea puede ser asignada a más de una persona. Observar que existe una restricción por cada tarea.

Vamos a implementar este modelo usando la interfaz Python de Gurobi. Primero es necesario importar módulo `gurobipy`:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

Consideraremos para este ejemplo un conjunto $I$ de cuatro personas y un conjunto $J$ de cuatro tareas. Definimos estos conjuntos como tuplas:

In [ ]:
# tupla con el conjunto ordenado de personas
I = ('Pablo', 'Carmen', 'Paula', 'Pedro', 'Jorge')
# tupla con el conjunto ordenado de tareas
J = ('base de datos', 'reportes', 'pruebas computacionales', 'formular modelo', 'conclusiones')

print ('I:= {}'.format(I))
print ('J:= {}'.format(J))

Las calificaciones de todas las posibles asignaciones se almacenan en un diccionario `calif` indexado por los pares ordenados $(i,j) \in I \times J$: 

In [ ]:
# puntajes de afinidad persona-tarea
calif = { 
    ('Pablo', 'base de datos') : 9, 
    ('Pablo', 'reportes') : 10, 
    ('Pablo', 'pruebas computacionales') : 8, 
    ('Pablo', 'formular modelo') : 9, 
    ('Pablo', 'conclusiones') : 7, 
    ('Carmen', 'base de datos') : 10, 
    ('Carmen', 'reportes') : 9, 
    ('Carmen', 'pruebas computacionales') : 8, 
    ('Carmen', 'formular modelo') : 8, 
    ('Carmen', 'conclusiones') : 10, 
    ('Paula', 'base de datos') : 10, 
    ('Paula', 'reportes') : 8, 
    ('Paula', 'pruebas computacionales') : 10, 
    ('Paula', 'formular modelo') : 9, 
    ('Paula', 'conclusiones') : 9, 
    ('Pedro', 'base de datos') : 9, 
    ('Pedro', 'reportes') : 8, 
    ('Pedro', 'pruebas computacionales') : 8, 
    ('Pedro', 'formular modelo') : 10,
    ('Pedro', 'conclusiones') : 9, 
    ('Jorge', 'base de datos') : 10, 
    ('Jorge', 'reportes') : 9, 
    ('Jorge', 'pruebas computacionales') : 7, 
    ('Jorge', 'formular modelo') : 9,
    ('Jorge', 'conclusiones') : 8} 

print(calif['Pablo', 'formular modelo'])

El siguiente paso es crear el objeto modelo:

In [ ]:
m = gp.Model('problema-asignacion')

Para crear las variables del modelo, podemos llamar a la función `addVar` dentro de dos lazos anidados que recorran sobre todos elementos de las tuplas `I` y `J`. Los objetos retornados por la función pueden almacenarse en un diccionario indexado por por tuplas $(i,j)$, de tal forma que `x[i,j]` corresponda a la variable $x_{ij}$, para todo $i \in I$ y $j \in J$.

In [ ]:
# Creación de variables
# Opción 1: Usar un lazo for 
# (no tomaremos esta opción)
# x = {}
# for i in I:
#     for j in J:
#         x[i,j]= m.addVar(vtype=GRB.BINARY, name="x[{},{}]".format(i,j))

Sin embargo, resulta más conveniente usar el método `addVars` de la clase `Model`. Este método permite crear una familia de variables indexadas por uno o más conjuntos iterables. En nuestro caso, las variables $x_{ij}$ están indexadas por todas las combinaciones de personas $i \in I$ y tareas $j \in J$. Por lo tanto, los dos primeros argumentos de la función serán `I` y `J`. El método `addVars` retorna un objeto de la clase `tupledict`, que es similar al diccionario de variables propuesto en la opción anterior, pero con funcionalidad adicional que resulta útil en la formulación del modelo.

Estudiaremos la clase `tupledict` con mayor detalle en el [Cuaderno 3](https://epnecuador-my.sharepoint.com/:u:/g/personal/luis_torres_epn_edu_ec/EYfkOmOZV9xFuAK72sdprKIB5RQUskS495OKbL5LDeAj_A?e=FsMpy4).

In [ ]:
# creamos variables x indexadas por I x J
x = m.addVars(I, J, vtype = GRB.BINARY, name="x")
print(type(x))
print()
m.update()
print(x)
print()
print([x[i,j].varName for i in I for j in J])

Los demás argumentos de la función son los mismos que los argumentos de `addVar`. El argumento `name` en este caso no fija el nombre de la variable, sino un prefijo que es completado con las posibles combinaciones de los índices.

Definimos ahora la función objetivo. Notar que debemos llamar al método `setObjective` del objeto modelo, y debemos pasarle como primer parámetro al sumatorio $\sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}$. Para implementar este sumatorio, podríamos usar una expresión generadora (*list comprehension*) para construir una lista con los términos de la suma, y llamar luego a la función `sum`: 

In [ ]:
# Construcción del sumatorio en la función objetivo
# Opción 1: Usar list comprehensions 
# (no tomaremos esta opción)
# m.setObjective(sum([calif[i,j]*x[i,j] for i in I for j in J]), GRB.MAXIMIZE)

Sin embargo, también podemos emplear una alternativa más concisa para definir el sumatorio, que consiste en llamar al método `prod` del objeto `x`:

In [ ]:
# la función objetivo es la suma de c_ij x_ij sobre i en I y j en J
m.setObjective(x.prod(calif, '*', '*'), GRB.MAXIMIZE)

Para definir cada una de las familias de restricciones, podríamos llamar al método `addConstr` desde dentro de un lazo. En su lugar, emplearemos el método `addConstrs` que permite crear una familia completa de restricciones utilizando una expresión generadora.

1. Cada persona puede tener asignada máximo una tarea: el primer parámetro de `addConstrs` es una expresión generadora que depende de la variable `i`, la misma que toma los valores de la tupla `I`. Para cada valor específico de `i`, el método `sum` de `x` se encarga de calcular la suma $\sum_{j \in J} x_{ij}$. El parámetro `asig_i` fija un nombre para la familia de restricciones. El nombre de cada restricción individual se genera automáticamente apendizando un sufijo con los elementos de `I`.

In [ ]:
# Cada persona puede tener asignada máximo una tarea
cI = m.addConstrs((x.sum(i, '*') <= 1 for i in I), "asig_i") 
m.update()
print(type(cI))
print()
print(cI)
print()
print([cI[i].getAttr('ConstrName') for i in I]) 

2. Cada tarea puede estar asignada máximo a una persona: utilizamos nuevamente el método `addConstrs`, cuyo primer parámetro es una expresión generadora que depende de la variable `j`, la misma que toma los valores de la tupla `J`. Para cada valor específico de `j`, el método `sum` de `x` se encarga de calcular la suma $\sum_{i \in I} x_{ij}$. El parámetro `asig_j` fija un nombre para la familia de restricciones.

In [ ]:
# Cada tarea puede estar asignada máximo a una persona
cJ = m.addConstrs((x.sum('*', j) <= 1 for j in J), "asig_j") 
m.update()
print(cJ)

Finalmente, resolvemos el modelo:

In [ ]:
m.optimize()

Terminada la optimización, podemos mostrar las soluciones. Notar que se usa la sintaxis `x[i,j]` para acceder a la variable $x_{ij}$, pues `x` es un diccionario cuyas claves son pares ordenados del conjunto $I \times J$:

In [ ]:
for i in I:
    for j in J:
        if x[i, j].x >= 0.99:
            print("Tarea {}: asignada a {}.".format(j, i))

## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [ ]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de asignación óptima
# EPN (2022)

import gurobipy as gp
from gurobipy import GRB
try:
    # tupla con el conjunto ordenado de personas
    I = ('Pablo', 'Carmen', 'Paula', 'Pedro')
    # tupla con el conjunto ordenado de tareas
    J = ('base de datos', 'reportes', 'pruebas computacionales', 'formular modelo')

    # puntajes de afinidad persona-tarea
    calif = { 
        ('Pablo', 'base de datos') : 9, 
        ('Pablo', 'reportes') : 10, 
        ('Pablo', 'pruebas computacionales') : 8, 
        ('Pablo', 'formular modelo') : 9, 
        ('Carmen', 'base de datos') : 10, 
        ('Carmen', 'reportes') : 9, 
        ('Carmen', 'pruebas computacionales') : 8, 
        ('Carmen', 'formular modelo') : 8, 
        ('Paula', 'base de datos') : 10, 
        ('Paula', 'reportes') : 8, 
        ('Paula', 'pruebas computacionales') : 10, 
        ('Paula', 'formular modelo') : 9, 
        ('Pedro', 'base de datos') : 9, 
        ('Pedro', 'reportes') : 8, 
        ('Pedro', 'pruebas computacionales') : 8, 
        ('Pedro', 'formular modelo') : 10} 

    m = gp.Model('problema-asignacion')

    # creamos variables x indexadas por I x J
    x = m.addVars(I, J, vtype = GRB.BINARY, name="x")
    
    # la función objetivo es la suma de c_ij x_ij sobre i en I y j en J
    m.setObjective(x.prod(calif, '*', '*'), GRB.MAXIMIZE)

    # Restricciones
    # Cada persona puede tener asignada máximo una tarea
    m.addConstrs((x.sum(i, '*') <= 1 for i in I), "asig_i") 

    # Cada tarea puede estar asignada máximo a una persona
    m.addConstrs((x.sum('*', j) <= 1 for j in J), "asig_j") 

    # Resolver el modelo
    m.optimize()

    # Mostrar la solución
    for i in I:
        for j in J:
            if x[i, j].x >= 0.99:
                print("Tarea {}: asignada a {}.".format(j, i))
            
except GurobiError as e:
    print('Código de error ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se encontró un error de atributo')